In [1]:
import tensorflow as tf
import tensorflow_io as tfio

2023-02-14 08:56:25.152831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
contents = 'audio_experiments/tronica_trimmed.wav'

In [12]:
# read the audio file into an IOTensor
audio = tfio.audio.AudioIOTensor(contents)
print(audio)

<AudioIOTensor: shape=[2425819       2], dtype=<dtype: 'int16'>, rate=48000>


In [13]:
tf.autograph.to_graph(audio)

ConversionError: converting <AudioIOTensor: shape=[2425819       2], dtype=<dtype: 'int16'>, rate=48000>: ValueError: Cannot apply autograph to a function that doesn't expose a __code__ object. If this is a @tf.function, try passing f.python_function instead.

In [10]:
audio2 = tf.audio.decode_wav(
    contents, desired_channels=2
)
print(audio2)

2023-02-14 09:07:39.321060: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Header mismatch: Expected RIFF but found tron


InvalidArgumentError: {{function_node __wrapped__DecodeWav_device_/job:localhost/replica:0/task:0/device:CPU:0}} Header mismatch: Expected RIFF but found tron [Op:DecodeWav]